# Setup the training environment

Check if a Nvidia GPU is availiable.

If you get `/bin/bash: line 1: nvidia-smi: command not found`, select the
**T4 GPU** from withing the **Change runtime type** menu option.
Executing the training on a CPU takes ages! 🐌

In [ ]:
!nvidia-smi

Wed May 21 13:04:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Install dependencies for the training process:

In [ ]:
!pip install pyyaml torch torchvision ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

Set some variables:

In [ ]:
import os

DRIVE = os.path.join('./drive', 'MyDrive') # The path to your Google Drive (including mount point)
DATASET_ZIP = os.path.join(DRIVE, 'ml/var-ai/datasets/soccer-ball-1408.zip')

In [ ]:
DATASET_ZIP

'./drive/MyDrive/ml/var-ai/datasets/soccer-ball-1364.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


from google.colab import drive

mount_point = 'drive'
drive.mount(mount_point)

Create a folder structure for the dataset:

In [ ]:
!mkdir -p \
    /content/dataset/{images,labels}/{train,val} \
    /content/.tmp

----

# Prepare the Dataset


Clear the `.tmp`-folder containing an old unsplit dataset, e.g. when using a new/different dataset:

In [ ]:
!rm -r /content/.tmp/*

rm: cannot remove '/content/.tmp/*': No such file or directory


Extract dataset:

In [ ]:
import zipfile

with zipfile.ZipFile(DATASET_ZIP, 'r') as zip_ref:
    zip_ref.extractall('/content/.tmp')

Clear the `dataset`-directory, useful if you want to create a different split:

In [ ]:
!rm \
    /content/dataset/images/train/* \
    /content/dataset/images/val/* \
    /content/dataset/labels/train/* \
    /content/dataset/labels/val/*

Specify split ratio:

In [ ]:
SPLIT_RATIO = 0.8

Split dataset:

In [ ]:
import shutil
import random

from pathlib import Path

output_dir = 'dataset'
image_dir = Path('.tmp/images')
label_dir = Path('.tmp/labels')

image_files = list(image_dir.glob('*.jpg'))
random.shuffle(image_files)

split_index = int(len(image_files) * SPLIT_RATIO)
train_images = image_files[:split_index]
val_images = image_files[split_index:]

def move_files(image_list, subset):
    for image_path in image_list:
        label_path = label_dir / (image_path.stem + '.txt')

        img_dst = Path(output_dir) / 'images' / subset
        lbl_dst = Path(output_dir) / 'labels' / subset
        img_dst.mkdir(parents=True, exist_ok=True)
        lbl_dst.mkdir(parents=True, exist_ok=True)

        shutil.copy(image_path, img_dst / image_path.name)
        if label_path.exists():
            shutil.copy(label_path, lbl_dst / label_path.name)

move_files(train_images, 'train')
move_files(val_images, 'val')

Create a `yaml`-configuration file for the the dataset:

In [ ]:
import yaml

DATASET_CONFIGURATION_PATH = 'dataset.yaml'

dataset_configuration = {
    'path': '/content/dataset',
    'train': 'images/train',
    'val': 'images/val',
    'nc': 1,
    'names': ['sports ball']
}

with open(DATASET_CONFIGURATION_PATH, 'w') as config:
    yaml.dump(dataset_configuration, config)

----

# Model Training/Fine-Tuning

Our biggest most current [dataset](https://drive.google.com/file/d/1iboKyJKFsHlWk4dnXQ8VEUtO6kBDrZ_h/view?usp=drive_link) contains 1364 labeld images (as of 06.05.2025)

### Paremeters

Find an overview of all avaliable parameters [here](https://docs.ultralytics.com/modes/train/#train-settings).

Specify it's output path (preferably Google Drive so you don't lose training data when **timing out**):

In [ ]:
OUTPUT_DIR = os.path.join(DRIVE, 'ml/var-ai/models')
MODEL = 'yolo11s.pt'
TIME = 4.5
PATIENCE = 10
CACHE = True
SEED = 42
PLOTS = True
IMAGE_SIZE = 1280

### Learning Rate

Learning rate of `0.005` for 1-10k images

In [ ]:
LEARNING_RATE = 0.005

### Batch Size

The [batch size](https://docs.ultralytics.com/guides/model-training-tips/#how-to-train-a-machine-learning-model) specifies the number of data samples a machine learning model processes in a single training iteration. For example `16` means there are 16 images processed at a time before performing a weight update. This value should be set to the maximum a GPU/Processing-Unit can handle in order to **utilize** the **whole hardware**. For yolo11 there is `-1` which selects the optimal batch size based on the used hardware. Setting this too high results in **memory errors**.

One could use *trail-and-error* to find the optimal batch size by increasing the
the number until mentioned memory errors occure.

In [ ]:
BATCH_SIZE = -1

### Layer Freezing

For [transfer learning](https://docs.ultralytics.com/yolov5/tutorials/transfer_learning_with_frozen_layers/) (aka. fine tuning) a good tradeoff is `10` layers, this means the 10 most inner layers are kept, while `0` means the whole model is retrained. A higher number results in **faster training**, as the later layers are not retrained. A **drawback** of a higher freeze is a **lower percision**.

In [ ]:
FREEZE = 10

### Naming

For consistency purposes, choose a name that represents some important parameters e.g.:

In [ ]:
!ls -la /content/drive/MyDrive/ml/var-ai/models | cut -d ' ' -f 10 | grep 'yolo'

yolo11m-sports-ball-img1920-freeze10-crash
yolo11s-sports-ball-img1280-freeze10
yolo11s-sports-ball-img1920-freeze10
yolo11s-sports-ball-img640-freeze10


In [ ]:
import sys
sys.exit("Please enter a new model name:")

SystemExit: Please enter a new model name:

In [ ]:
NAME = 'yolo11s-sports-ball-img1280-freeze10-1408'

### Training the yolo model

In [ ]:
import torch
from ultralytics import YOLO

# Check if cuda is avaliable, if not, don't even start the training!
device = torch.device('cuda' if torch.cuda.is_available() else exit)

model = YOLO(MODEL)
model = model.to(device)
model.train(
    data=DATASET_CONFIGURATION_PATH,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    device=device,
    freeze=FREEZE,
    lr0=LEARNING_RATE,
    project=OUTPUT_DIR,
    patience=PATIENCE,
    time=TIME,
    cache=CACHE,
    seed=SEED,
    plots=PLOTS,
    name=NAME)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 18.4M/18.4M [00:00<00:00, 270MB/s]


engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11s-sports-ball-img1280-freeze10-1408, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=./drive/MyDrive/ml/var-ai/models, 

100%|██████████| 755k/755k [00:00<00:00, 38.2MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     24/216      7.33G     0.6245     0.3623      0.866         10       1280: 100%|██████████| 87/87 [01:05<00:00,  1.32it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:06<00:00,  1.81it/s]

                   all        282        282      0.988      0.986      0.994      0.849
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 14, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



24 epochs completed in 0.501 hours.
Optimizer stripped from drive/MyDrive/ml/var-ai/models/yolo11s-sports-ball-img1280-freeze10-1408/weights/last.pt, 19.2MB
Optimizer stripped from drive/MyDrive/ml/var-ai/models/yolo11s-sports-ball-img1280-freeze10-1408/weights/best.pt, 19.2MB

Validating drive/MyDrive/ml/var-ai/models/yolo11s-sports-ball-img1280-freeze10-1408/weights/best.pt...
Ultralytics 8.3.141 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  27%|██▋       | 3/11 [00:02<00:06,  1.27it/s]